<a href="https://colab.research.google.com/github/Gail529/Mental-health/blob/main/tweet_emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import pandas as pd 
from tqdm import tqdm_notebook as tqdm 
from tqdm import trange
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [38]:
emo_lex=pd.read_excel('/content/NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.xlsx') # emotion lexicon 

In [40]:
emo_lex.head(5)

,English (en),Afrikaans (af),Albanian (sq),Amharic (am),Arabic (ar),Armenian (hy),Azeerbaijani (az),Basque (eu),Belarusian (be),Bengali (bn),Bosnian (bs),Bulgarian (bg),Catalan (ca),Cebuano (ceb),Chinese (Simplified) (zh-CN),Chinese (Traditional) (zh-TW),Corsican (co),Croatian (hr),Czech (cs),Danish (da),Dutch (nl),English (en).1,Esperanto (eo),Estonian (et),Finnish (fi),French (fr),Frisian (fy),Galician (gl),Georgian (ka),German (de),Greek (el),Gujarati (gu),Haitian Creole (ht),Hausa (ha),Hawaiian (haw),Hebrew (iw),Hindi (hi),Hmong (hmn),Hungarian (hu),Icelandic (is),...,Russian (ru),Samoan (sm),Scots Gaelic (gd),Serbian (sr),Sesotho (st),Shona (sn),Sindhi (sd),Sinhala (Sinhalese) (si),Slovak (sk),Slovenian (sl),Somali (so),Spanish (es),Sundanese (su),Swahili (sw),Swedish (sv),Tagalog (Filipino) (tl),Tajik (tg),Tamil (ta),Telugu (te),Thai (th),Turkish (tr),Ukrainian (uk),Urdu (ur),Uzbek (uz),Vietnamese (vi),Welsh (cy),Xhosa (xh),Yiddish (yi),Yoruba (yo),Zulu (zu),Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,aback,uit die veld geslaan,prapa,ተጭኗል,الى الوراء,շեղում,sanki,aback,ззаду,পশ্চাতে,aback,изненадан,sorpresa,aback,吓了一跳,嚇了一跳,aback,po krmi,překvapení,overrasket,verrast,aback,abako,kummitus,taaksepäin,abasourdi,ôfbrekke,aback,aback,zurück,προς τα πίσω,અચાનક,aback,aback,a hope,נבוך,अचंभे,aback,hátra,óvart,...,сзади,o se mea e leai,s an Iar-,абацк,back,aback,بي پناهه,අවාසනාවන්ත මෙන්ය,prekvapený,Nadaljuj,aback,detrás,aback,tamaa,en rygg,aback,бесабаб нест,அதிர்ச்சி,వెనక్కి,ผงะ,pupada,огидно,aback,shafqatsiz,buồn,aback,ukuphazamiseka,אַבאַק,aback,back,0,0,0,0,0,0,0,0,0,0
1,abacus,abakus,numërator,abacus,طبلية تاج,անբավարարություն,abacus,abako,абака,গণনা-যন্ত্রবিশেষ,abakus,Абак,àbac,abako,算盘,算盤,abacus,računaljka,počitadlo,abacus,telraam,abacus,abako,abacus,abacus,abaque,abacus,ábaco,აბაკუსი,Abakus,άβακας,એબાસ,abacus,ƙaura,ke kumu,אַבַּקוּס,अबेकस,abacus,golyós számológép,abacus,...,счеты,faʻailoga,abacus,абакус,abacus,abacus,عباسي,අවපීඩනය,počítadlo,abakus,abacus,ábaco,abacus,Abacus,kulram,abako,abacus,அபாகஸ்,అబాకస్,ลูกคิด,abaküs,абак,abacus,abakus,bàn tính,abacus,abacus,אַבאַקוס,abacus,Abacus,0,0,0,0,0,0,0,0,0,1
2,abandon,verlaat,braktis,ውጣ,تخلى,լքել,tərk et,bertan behera,адмовіцца ад,বর্জিত করা,napuštanje,изоставят,abandonar,biyaan,放弃,放棄,abbandunà,napustiti,opustit,opgive,in de steek laten,abandon,forlasi,loobuma,luopua,abandonner,fergees,abandonar,უარი თქე,verlassen,εγκαταλείπω,છોડી દો,abandone,watsi,haʻalele,לִנְטוֹשׁ,छोड़ देना,tso tseg,elhagyott,yfirgefa,...,отказаться от,lafoaia,thrèig,напустити,tlohela,siya,کلون,අතහැර දමන්න,opustiť,opusti,iska dhaaf,abandonar,ninggalkeun,kuacha,överge,abandunahin,тарк кунед,கைவிட,పరిత్యజించిన,ละทิ้ง,terketmek,відмовитися,چھوڑ دو,tark eting,từ bỏ,gadael,ukushiya,פאַרלאָזן,kọ silẹ,shiya,0,1,0,0,0,1,0,1,0,0
3,abandoned,verlate,braktisur,ተትቷል,مهجور,լքված,tərk etdi,abandonatutako,закінуты,পরিত্যক্ত,napušteno,изоставен,abandonat,gibiyaan,弃,棄,abbandunata,napušten,opuštěný,forladt,verlaten,abandoned,forlasita,mahajäetud,hylätty,abandonné,ferlitten,abandonado,მიტოვებული,verlassen,εγκαταλειμμένος,ત્યજી,abandone,watsi,haʻaleleʻia,נָטוּשׁ,त्यागा हुआ,tso tseg,elhagyatott,yfirgefinn,...,заброшенный,tuulafoaiina,thrèigte,напуштен,e tlohetsoe,asiyiwa,ڇڏيو ويو,අත්හැර දමා ඇත,opustený,opuščeni,ka tagay,abandonado,ditinggalkeun,kutelekezwa,övergiven,inabandunang,партофташуда,கைவிடப்பட்ட,రద్దు,ถูกปล่อยปละละเลย,terkedilmiş,покинутий,ترک کر دیا,tark etildi,bị bỏ rơi,wedi'u gadael,lahliwe,פארלאזן,abandoned,ishiyiwe,0,1,1,0,0,1,0,1,0,0
4,abandonment,verlating,braktisje,ማቋረጥ,التخلي عن,հրաժարվելով,ləğv,abandono,пакіданне,বিসর্জন,napuštanje,изоставяне,abandó,abandonment,放弃,放棄,abbandunamentu,napuštanje,opuštění,opgivelse,verlatenheid,abandonment,forlaso,loobumine,hylkääminen,abandon,ferplichting,abandono,მიტოვება,Aufgabe,εγκατάλειψη,પરિત્યાગ,abandon,watsi,ka haʻalele,נְטִישָׁה,संन्यास,kev nrauj,lemondás,yfirgefa,...,оставление,lafoaia,thrèigsinn,напуштање,ho lahla,kusiya,ڇڏيل,අත්හැර දැම

In [39]:
emolex_df=emo_lex[['English (en)','Positive','Negative','Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust']]
emotions=emolex_df.columns.drop('English (en)')
emolex_df.rename(columns={'English (en)':'word'},inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
del data["neutrality"]

In [4]:
tweets=pd.read_csv('/content/final_tweets.csv')

In [41]:
emolex_df

,word,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,aback,0,0,0,0,0,0,0,0,0,0
1,abacus,0,0,0,0,0,0,0,0,0,1
2,abandon,0,1,0,0,0,1,0,1,0,0
3,abandoned,0,1,1,0,0,1,0,1,0,0
4,abandonment,0,1,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
14177,zone,0,0,0,0,0,0,0,0,0,0
14178,zoo,0,0,0,0,0,0,0,0,0,0
14179,zoological,0,0,0,0,0,0,0,0,0,0
14180,zoology,0,0,0,0,0,0,0,0,0,0


In [42]:
tweets


,user,location,text,tweets
0,app_stayaway,Mumbai,People can also take advantage of your phone a...,people can also take advantage of your phone a...
1,rykorua,Italia,Christmas is a good time to start practicing g...,christmas is a good time to start practicing g...
2,allevin18,"Minneapolis, MN","""Driving in the Midst of #Depression"" https://...",driving in the midst of depression mindwand...
3,DulleyTopBooks,"North Vancouver, B.C. Canada",@BCGovNews May the Prince of Peace give us ALL...,may the prince of peace give u all peace of ou...
4,sadvibes43ver,NaN,Anyone else just pretending they’re ok?\n#sadt...,anyone else just pretending they re ok sadtw...
...,...,...,...,...
1995,Nem_RL,Tennessee,well looks like i am more fucked up than i tho...,well look like i am more fucked up than i thou...
1996,Saru_Faja,"Karachi,Pakistan","#IOK_ParadiseInPain\n42- Last year, Ms. Rehman...",iokparadiseinpain 42 last year ms rehman vi...
1997,PeteThornberg,"Cleveland, OH",@BillBindc Don’t forget newly diagnosed depres...,don t forget newly diagnosed depression anxi...
1998,BentsenElin,NaN,@sojac60 @ThatEricAlper I think he was diagnos...,i think he wa diagnosed with depression and bi...


In [7]:
del tweets['Unnamed: 0.1']

In [45]:
def tweet_emotion(tweet_df,col):
  new_tweet_df=tweet_df.copy()
  #new  dataframe that will store the emotion scores
  emo_df=pd.DataFrame(0,index=new_tweet_df.index, columns=emotions)
  lemmatizer=WordNetLemmatizer()

  
  with tqdm(total=len(list(new_tweet_df.iterrows()))) as pbar:
    for i ,row in new_tweet_df.iterrows(): 
        pbar.update(1) #update the progress bar
        tweet=word_tokenize(new_tweet_df.loc[i][col]) #the body of text for each individual tweet(row)

        for word in tweet:
            new=lemmatizer.lemmatize(word)#lemmatizing the words
            emo_score=emolex_df[emolex_df.word == new]     
            if not emo_score.empty:
                for emotion in list(emotions):
                    emo_df.at[i,emotion] += emo_score[emotion]


    new_tweet_df=pd.concat([new_tweet_df,emo_df] ,axis=1)
    return new_tweet_df

In [50]:
# splitting dataframe by row index 
tweet_df_1 = tweets.iloc[:1000,:] 
tweet_df_1

,user,location,text,tweets
0,app_stayaway,Mumbai,People can also take advantage of your phone a...,people can also take advantage of your phone a...
1,rykorua,Italia,Christmas is a good time to start practicing g...,christmas is a good time to start practicing g...
2,allevin18,"Minneapolis, MN","""Driving in the Midst of #Depression"" https://...",driving in the midst of depression mindwand...
3,DulleyTopBooks,"North Vancouver, B.C. Canada",@BCGovNews May the Prince of Peace give us ALL...,may the prince of peace give u all peace of ou...
4,sadvibes43ver,NaN,Anyone else just pretending they’re ok?\n#sadt...,anyone else just pretending they re ok sadtw...
...,...,...,...,...
995,ImanBenChaibah,"Dubai, United Arab Emirates",“I often thought of it as standing in a water ...,i often thought of it a standing in a water t...
996,ChooseTherapy,New York City,"@verv_inc sits down with mental health expert,...",sits down with mental health expert erin cant...
997,NevilleSouthall,wales,Arun! \n\nGreat to see you're back to your wri...,arun great to see youre back to your writing ...
998,NevilleSouthall,wales,Dad thinks we've got the cassettes upstairs. M...,dad think weve got the cassette upstairs migh...


In [51]:
final_df=tweet_emotion(tweet_df_1,'tweets')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [52]:
final_df.describe()

,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
count,1000.000000,1000.000000,1000.00000,1000.000000,1000.00000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000
mean,1.221000,0.935000,0.44800,0.810000,0.18200,0.640000,0.567000,0.62900,0.252000,0.637000
std,1.225423,1.121621,0.72237,0.938499,0.47021,0.908419,0.883341,0.87871,0.527991,0.849678
min,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,1.000000,1.000000,0.00000,1.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000
75%,2.000000,1.000000,1.00000,1.000000,0.00000,1.000000,1.000000,1.00000,0.000000,1.000000
max,8.000000,6.000000,4.00000,5.000000,5.00000,6.000000,5.000000,5.00000,5.000000,6.000000


In [53]:
tweet_df_2 = tweets.iloc[1001:,:] 
tweet_df_2

,user,location,text,tweets
1001,lunchboxdays13,she/her,@leftwing_agenda @pwexay @LPsMullet @hanniboll...,thissssssss i wasn t diagnosed with depres...
1002,InSayori,In House Because Of Quarantine,@meekonline @leorioasscheek I self diagnosed b...,i self diagnosed back in 2018 i always have a...
1003,btncl_jhai,NaN,Minsan pinangarap ko talagang makilala ako ng ...,minsan pinangarap ko talagang makilala ako ng ...
1004,idiotlizzzy,-she/her-,"@twpissnap rq im gonna pop in here, I'm diagno...",rq im gonna pop in here im diagnosed depressi...
1005,theeplatano,"Houston, TX",1st generation graduate.\nBattled depression.\...,1st generation graduate battled depression d...
...,...,...,...,...
1995,Nem_RL,Tennessee,well looks like i am more fucked up than i tho...,well look like i am more fucked up than i thou...
1996,Saru_Faja,"Karachi,Pakistan","#IOK_ParadiseInPain\n42- Last year, Ms. Rehman...",iokparadiseinpain 42 last year ms rehman vi...
1997,PeteThornberg,"Cleveland, OH",@BillBindc Don’t forget newly diagnosed depres...,don t forget newly diagnosed depression anxi...
1998,BentsenElin,NaN,@sojac60 @ThatEricAlper I think he was diagnos...,i think he wa diagnosed with depression and bi...


In [54]:
final_df_2=tweet_emotion(tweet_df_2,'tweets')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [55]:
final_df_2.describe()

,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
count,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000
mean,0.587588,1.621622,0.496496,0.564565,0.199199,0.694695,0.240240,1.286286,0.128128,0.391391
std,0.800855,1.134702,0.684598,0.733937,0.473082,0.899723,0.524302,0.897627,0.365879,0.625098
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,1.000000,2.000000,1.000000,1.000000,0.000000,1.000000,0.000000,2.000000,0.000000,1.000000
max,4.000000,7.000000,4.000000,4.000000,3.000000,6.000000,4.000000,5.000000,2.000000,4.000000


In [56]:
tweet_emotions_df=final_df.append(final_df_2, ignore_index=True, sort=False)

In [57]:
tweet_emotions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user          1999 non-null   object
 1   location      1547 non-null   object
 2   text          1999 non-null   object
 3   tweets        1999 non-null   object
 4   Positive      1999 non-null   int64 
 5   Negative      1999 non-null   int64 
 6   Anger         1999 non-null   int64 
 7   Anticipation  1999 non-null   int64 
 8   Disgust       1999 non-null   int64 
 9   Fear          1999 non-null   int64 
 10  Joy           1999 non-null   int64 
 11  Sadness       1999 non-null   int64 
 12  Surprise      1999 non-null   int64 
 13  Trust         1999 non-null   int64 
dtypes: int64(10), object(4)
memory usage: 218.8+ KB


In [58]:
tweet_emotions_df.to_csv('tweet_emotion_scores.csv')